In [2]:
import warnings

warnings.filterwarnings(action='ignore')

## 3. 평가 (147p)

### 분류의 성능 평가 지표
1. 정확도(Accuracy)

2. 오차행렬(Confusion Matrix)

3. 정밀도(Precision)

4. 재현율(Recall)

5. F1 스코어

6. ROC AUC

이진 분류의 경우에는 데이터의 구성에 따라 ML 모델의 성능을 왜곡할 수 있기 때문에 정확도 수치 하나만 가지고 성능을 평가하지 않는다.

타이타닉의 경우에는, 단지 성별 조건 하나만 가지고 결정하는 별거 아닌 알고리즘도 높은 정확도를 나타내는 상황이 발생한다.

### BaseEstimator
Costomized 형태의 Estimator를 개발자가 생성할 수 있게 됨

In [6]:
## 단순히 Sex가 1이면 0, 그렇지 않으면 1로 예측하는 매우 단순한 Classifier
from sklearn.base import BaseEstimator
import numpy as np

class MyDummyClassifier(BaseEstimator):
    # fit() 메서드는 아무것도 학습하지 않음
    def fit(self, X, y=None):
        pass
    
    # predict() 메서드는 단순히 Sex에 따라 0과 1을 반환함
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1: pred[i] = 0
            else: pred[i] = 1
                
        return pred

In [7]:
np.zeros((10, 1))

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

### MyDummyClassifier를 이용한 타이타닉 생존자 예측

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [16]:
titanic = pd.read_csv('C:/Users/user/seaborn-data/titanic_train_kaggle.csv')

sel = titanic.columns.drop('Survived')

X = titanic[sel]
y = titanic['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.2, random_state=0)

X = transform_features(X)
X

NameError: name 'transform_features' is not defined